In [1]:
import pandas as pd
import numpy as np
import os
import random
from scipy.stats import pearsonr,spearmanr
from scipy import stats

In [2]:
# fdr multiple testing correction
from statsmodels.stats import multitest

def fdr_correction(P):
    size = P.shape
    temp_p = P.flatten()
    Ps = multitest.multipletests(temp_p,alpha=0.05,method='fdr_bh')
    P_corrected = Ps[1].reshape(size)

    return P_corrected

/data/sliu/anaconda3/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [3]:
# the function of calculating Jaccard index
from sklearn.metrics import confusion_matrix,jaccard_score
def Jaccard_index(data1,data2,t=None):
    s = data1.shape
    odata1 = np.zeros((s[0],2))
    for i in range(s[0]):
        tmp_l = []
        for j in range(s[1]):
            corr_map1 = data1[i,j,:,:] #Note from 1_prep_UKdata the firsst index is the subsample size, the second is the iteration, the third is the effect 'number, and the fourth is indexing r or p value (0 or 1).
            corr_map2 = data2[i,j,:,:] 
            # the thresholds to select significant brain associations
            if t == 'fdr':
                correct_P1 = fdr_correction(corr_map1[:,1])
                correct_P2 = fdr_correction(corr_map2[:,1])
                
                l1 = np.where(correct_P1 < 0.05)[0].tolist()
                l2 = np.where(correct_P2 < 0.05)[0].tolist()
            elif t == 'bonferroni':
                pt = 0.05/s[2]
                l1 = np.where(corr_map1[:,1] < pt)[0].tolist()
                l2 = np.where(corr_map2[:,1] < pt)[0].tolist()
            else:
                l1 = np.where(corr_map1[:,1] < float(t))[0].tolist()
                l2 = np.where(corr_map2[:,1] < float(t))[0].tolist()
                
            seqs = np.zeros((s[2],2))
            seqs[l1,0] = 1
            seqs[l2,1] = 1
            if len(l1) == 0 and len(l2) == 0:
                confusion_matrix1 = np.zeros((2,2))
                confusion_matrix1[0,0] = s[2]
            else:
                confusion_matrix1 = confusion_matrix(seqs[:,0],seqs[:,1])
                
            confusion_matrix2 = np.zeros((2,2))
            confusion_matrix2[0,0] = (confusion_matrix1[0,0] + confusion_matrix1[0,1])*(confusion_matrix1[0,0] + confusion_matrix1[1,0])
            confusion_matrix2[1,1] = (confusion_matrix1[1,1] + confusion_matrix1[0,1])*(confusion_matrix1[1,1] + confusion_matrix1[1,0])
            confusion_matrix2[0,1] = (confusion_matrix1[1,1] + confusion_matrix1[0,1])*(confusion_matrix1[0,0] + confusion_matrix1[0,1])
            confusion_matrix2[1,0] = (confusion_matrix1[1,1] + confusion_matrix1[1,0])*(confusion_matrix1[0,0] + confusion_matrix1[1,0])
            
            n1 = confusion_matrix1[1,1] + confusion_matrix1[1,0] + confusion_matrix1[0,1]
            if n1 == 0:
                S = 0
            else:
                S = confusion_matrix1[1,1]/ n1
            
            n2 = confusion_matrix2[1,1] + confusion_matrix2[1,0] + confusion_matrix2[0,1]
            if n2 == 0:
                ES = 0
            else:
                ES = confusion_matrix2[1,1]/n2
            
            dd = (S - ES)/(1-ES)
            tmp_l.append(dd)
                
        odata1[i,0] = np.mean(tmp_l)
        odata1[i,1] = np.std(tmp_l)
    return odata1

## statistical_analysis_Jaccard: The main function to calculate the Jaccard index
### input parameters: file_path, npy_file1, npy_file2, *t*, mytype, output_path
**file_path:** The folder that includes the npy files (The bootstrapped correlations)

**npy_file1:** The npy file obtained from subsample1

**npy_file2:** The npy file obtained from subsample2

**t:** The significance thresholds: p<0.05, p<0.01, fdr_p<0.05, fdr_bonferroni<0.05. For example, *t* = '0.05', '0.01','fdr',or 'bonferroni'

**mytype:** The imaging type: "CSA", "CT", or "FC"

**output_path:** The folder where the results of Jaccard index are saved 

In [10]:
def statistical_analysis_Jaccard(file_path,npy_file1,npy_file2,t,mytype,output_path):
    file_path = 'boostrap_correlations/'
    files = os.listdir(file_path)
    for f in files:
        file_path1 = os.path.join(file_path,f) + '/'+ npy_file1
        file_path2 = os.path.join(file_path,f) + '/'+ npy_file2
        random_data1 = np.load(file_path1)
        random_data2 = np.load(file_path2)

        reliability = Jaccard_index(random_data1,random_data2,t=t)
        if not os.path.exists(output_path+f):
            os.mkdir(output_path+f)
        file_name = output_path+f+'/'+mytype+'_Jaccard_index_'+t+'.csv'
        data = pd.DataFrame(data=reliability)
        data.to_csv(file_name,index=False)
    

In [7]:
# runing the main function to estimate Jaccard index
ts = ['0.05', '0.01','fdr','bonferroni']
for t in ts:
    print("The significance level is: ", t)
    statistical_analysis_Jaccard("boostrap_correlations/","random_data_CSA1.npy","random_data_CSA2.npy",\
                         t,"CSA","Jaccard/")
    statistical_analysis_Jaccard("boostrap_correlations/","random_data_CT1.npy","random_data_CT2.npy",\
                         t,"CT","Jaccard/")
    statistical_analysis_Jaccard("boostrap_correlations/","random_data_FC1.npy","random_data_FC2.npy",\
                         t,"FC","Jaccard/")

The significance level is:  0.05
The significance level is:  0.01
The significance level is:  fdr
The significance level is:  bonferroni


Below the calculate_ICC function is defined without clarifying what different variables refer to. 

It is worth noting that the intraclass correlation coefficient is used as a reliability measure similar to test-retest (discovery and replication). Here the 'raters' are the discovery (A) and replication (B) datasets respsecitvely. A 'rating' refers to a given effect size for each rater. The 'subjects' or 'targets' are here each effect size, r.

In [ ]:
# the function of calculating Intraclass Correlation Coefficient
import pingouin as pg
from scipy import stats

def calculate_ICC(data1,data2,t = None):
    subsampling_times = data1.shape[0] #nBinsize
    random_num = data1.shape[1] #nIterations
    ICC_data = np.zeros((subsampling_times,2)) 
    t = float(t) #Proportion of all effects
    region_num = round(t * data1.shape[2])
    for i in range(subsampling_times):
        print(i)
        temp_icc = []
        for j in range(random_num):
            
            my_targets = []
            my_raters = []
            my_ratings = []
            
            tdata1 = data1[i,j,:,:] #Pulls discovery sample effect sizes (r) and p values, for a given binsize (i) and iteration (j).
            tdata2 = data2[i,j,:,:] #Pulls replication sample...
            zdata1 = stats.zscore(tdata1[:,0])
            zdata2 = stats.zscore(tdata2[:,0])
            argindexs1 = np.argsort(tdata1[:,1])[:region_num] #Picks the smallest p-values up to region_num for each t.
            tmp1 = zdata1[argindexs1]
            tmp2 = zdata2[argindexs1]
            

            for k in range(region_num): #I think this is for k in 1,2,3,...,region_num.
                my_targets.append(k+1)
                my_raters.append('A')
                my_ratings.append(tmp1[k])
  
                my_targets.append(k+1)
                my_raters.append('B')
                my_ratings.append(tmp2[k])
        #Creating a my_df_data dataframe for using the pg.intraclass_corr function.
            my_df_data = pd.DataFrame(data=my_targets,columns=['region']) #The my_targets column will be a list 1,1,2,2,3,3,...,region_num,region_num.
            my_df_data['random_time'] = my_raters #Poor naming scheme. This is a list of As and Bs.
            my_df_data['corr'] = my_ratings #This is the list of the top t% effect sizes, alternating from discovery and replication.

            # ICC2: A random sample of k raters rate each target. The measure is one of absolute agreement in the ratings.
            my_icc = pg.intraclass_corr(data=my_df_data, targets='region', raters='random_time', ratings='corr')['ICC'].values[1]
            temp_icc.append(my_icc)
    
        ICC_data[i,0] = np.mean(temp_icc) #averaging ICC over iterations
        ICC_data[i,1] = np.std(temp_icc) #std of ICCs.
    return ICC_data

## statistical_analysis_ICC: The main function to calculate the ICC
### input parameters: file_path, npy_file1, npy_file2, *t*, mytype, output_path
**file_path:** The folder that includes the npy files (The bootstrapped correlations)

**npy_file1:** The npy file obtained from subsample1

**npy_file2:** The npy file obtained from subsample2

**t:** The proportion of correlations to calculate the ICC: 0.1,0.15,0.2,0.25,0.5,1

**mytype:** The imaging type: "CSA", "CT", or "FC"

**output_path:** The folder where the results of Jaccard index are saved 

In [9]:
def statistical_analysis_ICC(file_path,npy_file1,npy_file2,t,mytype,output_path):
    file_path = 'boostrap_correlations/'
    files = os.listdir(file_path)
    for f in files:
        file_path1 = os.path.join(file_path,f) + '/'+ npy_file1
        file_path2 = os.path.join(file_path,f) + '/'+ npy_file2
        random_data1 = np.load(file_path1)
        random_data2 = np.load(file_path2)
        
        reliability = calculate_ICC(random_data1,random_data2,t=t)
        if not os.path.exists(output_path+f):
            os.mkdir(output_path+f)
        file_name = output_path+f+'/'+mytype+'_ICC_'+t+'.csv'
        data = pd.DataFrame(data=reliability)
        data.to_csv(file_name,index=False)

In [ ]:
# runing the main function to estimate ICC
ts = ['0.1', '0.15','0.2','0.25','0.5','1']
for t in ts:
    print("The ICC proportion is: ", t)
    statistical_analysis_ICC("boostrap_correlations/","random_data_CSA1.npy","random_data_CSA2.npy",\
                         t,"CSA","ICC/")
    statistical_analysis_ICC("boostrap_correlations/","random_data_CT1.npy","random_data_CT2.npy",\
                         t,"CT","ICC/")
    statistical_analysis_ICC("boostrap_correlations/","random_data_FC1.npy","random_data_FC2.npy",\
                         t,"FC","ICC/")